In [ ]:
import cv2
import numpy as np
import torch
from tkinter import Tk, filedialog, Label, Button, Canvas, PhotoImage
from PIL import Image, ImageTk
#pip install opencv-python pillow torch torchvision matplotlib ultralytics


In [16]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\mahes/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-21 Python-3.11.0 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:

# Function classify car color
def classify_car_color(image, box):
    x1, y1, x2, y2 = map(int, box)
    cropped_img = image[y1:y2, x1:x2]
    avg_color = np.mean(cropped_img, axis=(0, 1))
    if avg_color[2] > avg_color[0]:  
        return 'Red'
    elif avg_color[0] > avg_color[2]:  
        return 'Blue'
    else:
        return 'Other'

In [22]:
# Function to process image
def process_image(image_path):
    image = cv2.imread(image_path)
    results = model(image)

    
    detections = results.xyxy[0].numpy()
    person_count = 0
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        label = model.names[int(cls)]
        if label == 'car':
            color = classify_car_color(image, [x1, y1, x2, y2])
            color_rect = (255, 0, 0) if color == 'Blue' else (0, 0, 255)
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color_rect, 2)
        elif label == 'person':
            person_count += 1

    
    cv2.putText(image, f"People Count: {person_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    return image, person_count


In [23]:
# GUI
class CarColorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Car Color Detection")
        self.canvas = Canvas(root, width=800, height=600)
        self.canvas.pack()

        self.label = Label(root, text="Upload an Image")
        self.label.pack()

        self.upload_button = Button(root, text="Upload", command=self.upload_image)
        self.upload_button.pack()

    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            processed_img, person_count = process_image(file_path)
            processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
            processed_img = Image.fromarray(processed_img)
            processed_img = ImageTk.PhotoImage(processed_img)

            self.canvas.create_image(0, 0, anchor="nw", image=processed_img)
            self.canvas.image = processed_img
            self.label.config(text=f"People Count: {person_count}")


root = Tk()
app = CarColorApp(root)
root.mainloop()